In [100]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastf1
import plotly.express as px
import fastf1.plotting
import datetime
import requests
from datetime import timedelta

fastf1.Cache.enable_cache('../f1_cache')  

In [113]:
selected_year = 2021
fastf1_session = fastf1.get_session(selected_year, 'Bahrain', 'Race')
fastf1_session.load(telemetry=True, laps=True, weather=False)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.2.3]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['44', '33', '77', '4', '11', '16', '3', '55', '22', '18', '7', '99', '31', '63', '5', '47', '10', '6', '14', '9']


# LapTime vs LapNumber

In [37]:
selected_drivers = ['HAM', 'RIC']
lap_time_number = fastf1_session.laps[['Driver', 'LapTime', 'Compound', 'TyreLife', 'LapNumber']].copy()
lap_time_number = lap_time_number[lap_time_number['Driver'].isin(selected_drivers)]
lap_time_number['LapTime'] = lap_time_number['LapTime'].fillna(timedelta(0))
lap_time_number['Compound'] = lap_time_number['Compound'].fillna("")
lap_time_number['Year'] = selected_year
lap_time_number['TyreLife'] = lap_time_number['TyreLife'].fillna(-1)
lap_time_number['LapTime'] = pd.to_numeric(lap_time_number['LapTime'])/1000000
lap_time_number_cov = lap_time_number.to_dict('records')
final_dict = {}
for data in lap_time_number_cov:
    primary_key = data['Driver']
    data[primary_key] = data.pop('LapTime')

    # color
    driv_lap = fastf1_session.laps.pick_driver(primary_key).pick_fastest()
    color = fastf1.plotting.team_color(driv_lap['Team'])
    data['color'] = color

    if primary_key not in final_dict:
        final_dict[primary_key] = [data]
    else:
        final_dict[primary_key].append(data)

# for driv in final_dict:
#     thelist = final_dict[driv]
#     thelist[:] = [d for d in thelist if d.get(driv) != 0]

# Team Color Setup

In [93]:
team_color_pair = {'Red Bull Racing': '#0600ef',
 'Williams': '#ADD8E6',
 'Mercedes': '#900000',
 'AlphaTauri': '#3e454d',
 'Alpine': '#0090ff',
 'Ferrari': '#dc0000',
 'Alpha Romeo': '#900000',
 'Haas': '#ffffff',

 'McLaren': '#ffa112',

 'Aston Martin': '#006f62',
}

In [95]:
selected_drivers = ['VER', 'PER', 'RUS', 'LAT', 'BOT', "HAM", 'GAS', 'ALO', 'OCO', 'SAI', 'LEC', 'RAI', 'GIO', 'MSC', 'TSU', 'MAZ', 'NOR', 'RIC', 'VET', 'STR', 'HUL', 'MAG', 'ZHO']
for driver in selected_drivers:
    driv_lap = fastf1_session.laps.pick_driver(driver).pick_fastest()
    print(driv_lap['Team'])
    # try:
    #     team = driv_lap['Team']
    #     color = team_color_pair[team]
    # except:
    #     color = "#fffff"

Red Bull Racing
Red Bull Racing
Mercedes
Williams
Alfa Romeo
Mercedes
AlphaTauri
Alpine
Alpine
nan
Ferrari
nan
nan
Haas F1 Team
AlphaTauri
nan
McLaren
McLaren
Aston Martin
Aston Martin
nan
Haas F1 Team
Alfa Romeo


/Users/harsh/miniconda3/envs/f1/lib/python3.9/site-packages/fastf1/core.py:2082: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  super().__init__(*args, **kwargs)


# Speed vs Distance

In [116]:
selected_drivers = ['HAM', 'ALO', 'LAT']

driv_tel = {}
for driver in selected_drivers:
    driv_lap = fastf1_session.laps.pick_driver(driver).pick_fastest()
    color = fastf1.plotting.team_color(driv_lap['Team'])
    driv_tel[driver] = driv_lap.get_car_data().add_distance(
    )[['Speed', 'Distance', 'RPM', 'nGear', 'Throttle', 'Brake', 'DRS']]
    driv_tel[driver]['Brake'] = driv_tel[driver]['Brake'].astype(int)
    driv_tel[driver]['color'] = color
    # driv_tel[driver] = driv_tel[driver].to_dict('records')

In [36]:
f1_season = fastf1.get_event_schedule(int(2021))
event_records = f1_season[['RoundNumber', 'EventName']]
round_number = pd.DataFrame(event_records)[event_records['EventName'] == 'Monaco Grand Prix']['RoundNumber']

In [43]:
type(round_number.iloc[0])

numpy.int64

In [21]:
driver_list = [x[:3] for x in requests.get(
            "http://ergast.com/api/f1/{}/{}/drivers".format(2021, round_number))]

In [24]:
requests.get("http://ergast.com/api/f1/{}/{}/drivers".format(2021, round_number)).text

'<html>\r\n  <head>\r\n    <title>Error</title>\r\n  </head>\r\n  <body>\r\n    <h3>Bad Request</h3>\r\n  </body>\r\n</html>\r\n'